<a href="https://colab.research.google.com/github/lucas-t-reis/CUDA/blob/master/notebooks/Professioanl_CUDA_programming_exercises.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- Checking for Nvidia **CUDA toolkit**
- Checking for Nvidia Devices (enabled in Runtime type)

In [0]:
!which nvcc
!ls -l /dev/nv*

/usr/local/cuda/bin/nvcc
crw-rw-rw- 1 root root 195,   0 Jan  7 21:21 /dev/nvidia0
crw-rw-rw- 1 root root 195, 255 Jan  7 21:21 /dev/nvidiactl
crw-rw-rw- 1 root root 247,   0 Jan  7 21:21 /dev/nvidia-uvm
crw-rw-rw- 1 root root 247,   1 Jan  7 21:21 /dev/nvidia-uvm-tools


### Installing plugin to allow running CUDA C/C++ - by [andreinechaev](https://github.com/andreinechaev)

In [2]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-eco90dxc
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-eco90dxc
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-cp36-none-any.whl size=4307 sha256=1bd7ab19e865e43a8541354968aabb624386150e86dc868e9058d20fc5ed902e
  Stored in directory: /tmp/pip-ephem-wheel-cache-f6vba6l9/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin
created output directory at /content/src
Out bin /content/result.out


# Hello World - Example 1

In [0]:
%%cu
#include <stdio.h>

// __global__ qualifier to denote a function called by CPU and executed by GPU    
__global__ void helloWorldGPU() {
    
    int gpu_tid = threadIdx.x;
    printf("Hello World from GPU thread %d!\n", gpu_tid);
}

int main() {
    printf("Hello World from CPU!\n");
    
    // Kernel
    helloWorldGPU <<< 1,10 >>> ();

    cudaDeviceReset();
}

Hello World from CPU!
Hello World from GPU thread 0!
Hello World from GPU thread 1!
Hello World from GPU thread 2!
Hello World from GPU thread 3!
Hello World from GPU thread 4!
Hello World from GPU thread 5!
Hello World from GPU thread 6!
Hello World from GPU thread 7!
Hello World from GPU thread 8!
Hello World from GPU thread 9!



## Host based array summation (unfinished)
pg.28

In [5]:
%%cu
#include <stdlib.h>
#include <time.h>
#include <string.h>

void h_sum(float *a, float *b, float *c, const int N) {
    for(int i=0; i<N; i++) 
        c[i] = a[i] + b[i];
}

void fillArray(float *arr, int size) {
    
    // Getting random number based on current time
    time_t t;
    srand( (unsigned int) time(&t));

    for(int i=0; i<size; i++)
      arr[i] = (rand()%0xFF)/10.0f; // modulo 255

}

int main () {
    
    int n = 1024;
    size_t nBytes = n * sizeof(float);


    float *h_a, *h_b, *h_c; // h_ stands for host data
    float *d_a, *d_b, *d_c; // d_ stands for device data

    h_a = (float *) malloc(nBytes);
    h_b = (float *) malloc(nBytes);
    h_c = (float *) malloc(nBytes);

    cudaMalloc( (float**) &d_a, nBytes);
    cudaMalloc( (float**) &d_b, nBytes);
    cudaMalloc( (float**) &d_c, nBytes);
    
    fillArray(h_a, n);
    fillArray(h_b, n);

    cudaMemcpy(d_a, h_a, nBytes, cudaMemcpyHostToDevice); // params (dest, src, total data bytes, direction)
    cudaMemcpy(d_b, h_b, nBytes, cudaMemcpyHostToDevice);

    // Host sum
    h_sum(d_a, d_b, d_c, n);


    free(h_a);
    free(h_b);
    free(h_c);

    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);
    
}